In [1]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers

import torch
from torch import nn

import torchvision

import numpy as np
import matplotlib.pyplot as plt

<b>TensorFlow/Keras:</b>
``` python
tf.keras.layers.GroupNormalization(
    groups=32, axis=-1, epsilon=0.001, center=True, scale=True,
    beta_initializer='zeros', gamma_initializer='ones',
    beta_regularizer=None, gamma_regularizer=None,
    beta_constraint=None, gamma_constraint=None, **kwargs)
```
Group Normalization divides the channels into groups and computes within each group the mean and variance for normalization. Empirically, its accuracy is more stable than batch norm in a wide range of small batch sizes, if learning rate is adjusted linearly with batch sizes.

Relation to Layer Normalization: If the number of groups is set to 1, then this operation becomes nearly identical to Layer Normalization (see Layer Normalization docs for details).

Relation to Instance Normalization: If the number of groups is set to the input dimension (number of groups is equal to number of channels), then this operation becomes identical to Instance Normalization. You can achieve this via groups=-1.

<b>PyTorch:</b>
``` python
torch.nn.GroupNorm(num_groups, num_channels, eps=1e-05, affine=True, device=None, dtype=None)
```
Applies Group Normalization over a mini-batch of inputs.
This layer implements the operation as described in the paper Group Normalization.

$y=\dfrac{x-E[x]}{\sqrt(Var[x]+\epsilon)}\ast \gamma + \beta$

The input channels are separated into num_groups groups, each containing num_channels / num_groups channels. num_channels must be divisible by num_groups. The mean and standard-deviation are calculated separately over the each group. $\gamma$ and $\beta$ are learnable per-channel affine transform parameter vectors of size num_channels if affine is True. The standard-deviation is calculated via the biased estimator, equivalent to torch.var(input, unbiased=False).

This layer uses statistics computed from input data in both training and evaluation modes.

</pre>

# When parameters ($\beta$ and $\gamma$) are initialized as 0 and 1, respectively

In [89]:
# define random inputs
input_torch = torch.randn(1, 6, 2, 2) # batch, channel, h, w
input_tf = tf.convert_to_tensor(input_torch.permute((0,2,3,1))) # batch, h, w, channel

# define group normalization layer
norm_tf = layers.GroupNormalization(groups=3, axis=-1, epsilon=1e-04)
norm_torch = nn.GroupNorm(num_groups=3,num_channels=6, eps=1e-04)

out1 = norm_tf(input_tf)
out2 = norm_torch(input_torch)
# out3 = norm_torch2(input_torch)

print('tensorflow output: \n', out1.numpy().reshape(-1))
print('pytorch output: \n', out2.detach().permute((0,2,3,1)).numpy().reshape(-1))
# print('pytorch output2: \n', out3.detach().numpy().reshape((-1)))

tensorflow output: 
 [ 0.84968716 -0.5196075  -1.7287757  -0.2732721  -2.4594684   0.9767574
 -1.4404817  -0.86852103  0.91459125  1.0332913  -0.16894978  0.2635165
  1.4574002   0.18470833 -0.3066881   0.32460397  0.754851    0.5058085
  1.1588684  -0.8220539   1.2080045  -1.1717552  -0.11549518  0.24297996]
pytorch output: 
 [ 0.84968716 -0.51960754 -1.728776   -0.2732722  -2.4594686   0.97675747
 -1.4404817  -0.86852103  0.9145913   1.0332915  -0.16894983  0.2635165
  1.4574002   0.18470833 -0.3066882   0.32460397  0.7548511   0.5058085
  1.1588684  -0.8220539   1.2080046  -1.1717553  -0.11549523  0.24297994]


In [90]:
# see the parameters
print('tensorflow:')
display(norm_tf.weights)

print('pytorch:')
display(norm_torch.weight, norm_torch.bias)

tensorflow:


[<tf.Variable 'group_normalization_42/gamma:0' shape=(6,) dtype=float32, numpy=array([1., 1., 1., 1., 1., 1.], dtype=float32)>,
 <tf.Variable 'group_normalization_42/beta:0' shape=(6,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0.], dtype=float32)>]

pytorch:


Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)

Parameter containing:
tensor([0., 0., 0., 0., 0., 0.], requires_grad=True)

# What if the same random weights are used?


In [92]:
n_batch = 2
n_channel = 6
n_group = 3
n_size = 3 # height & width of input

gamma = torch.randn(n_channel, requires_grad=True)
beta = torch.randn(n_channel, requires_grad=True)

# define group normalization layer
norm_tf = layers.GroupNormalization(groups=n_group, axis=-1, epsilon=1e-04)
norm_torch = nn.GroupNorm(num_groups=n_group,num_channels=n_channel, eps=1e-04)

# feed the input first (for some reason, norm_tf did not have weights before it receive input)
input_torch = torch.randn(n_batch, n_channel, n_size, n_size) # batch, channel, h, w
input_tf = tf.convert_to_tensor(input_torch.permute((0,2,3,1))) # batch, h, w, channel

out1 = norm_tf(input_tf)
out2 = norm_torch(input_torch)

# change the weights
norm_tf.set_weights([gamma.detach().numpy(), beta.detach().numpy()])

norm_torch.weight = torch.nn.Parameter(gamma) # gamma
norm_torch.bias = torch.nn.Parameter(beta) # beta

# print the changed weights
print('tf gamma: \n',norm_tf.weights[0])
print('torch gamma: \n',norm_torch.weight)
print('tf beta: \n',norm_tf.weights[1])
print('torch beta: \n',norm_torch.bias)

# define random inputs again
input_torch = torch.randn(n_batch, n_channel, n_size, n_size) # batch, channel, h, w
input_tf = tf.convert_to_tensor(input_torch.permute((0,2,3,1))) # batch, h, w, channel

out1 = norm_tf(input_tf)
out2 = norm_torch(input_torch)

print('tensorflow output: \n', out1.numpy().reshape((-1)))
print('pytorch output: \n', out2.detach().permute((0,2,3,1)).numpy().reshape((-1)))

tf gamma: 
 <tf.Variable 'group_normalization_44/gamma:0' shape=(6,) dtype=float32, numpy=
array([ 1.1889397 ,  0.43409932, -0.07469586,  0.18262534, -0.74273956,
        0.5033717 ], dtype=float32)>
torch gamma: 
 Parameter containing:
tensor([ 1.1889,  0.4341, -0.0747,  0.1826, -0.7427,  0.5034],
       requires_grad=True)
tf beta: 
 <tf.Variable 'group_normalization_44/beta:0' shape=(6,) dtype=float32, numpy=
array([-0.16885795,  1.9923154 ,  0.19933347,  0.66141623, -1.5776755 ,
       -0.08880939], dtype=float32)>
torch beta: 
 Parameter containing:
tensor([-0.1689,  1.9923,  0.1993,  0.6614, -1.5777, -0.0888],
       requires_grad=True)
tensorflow output: 
 [ 1.0477364   2.7123933   0.2418355   0.72782457 -2.1019793   0.73462945
  0.467426    2.1588924   0.27451015  0.9668887  -1.4655087   0.23099075
  0.46032774  1.4361737   0.09584648  0.65685666 -1.3608897   0.80878407
  0.13111523  1.4709439   0.20161298  0.456112   -1.1804442  -0.67813414
 -1.2264485   1.7408135   0.29957572